In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Anxiety_disorder"
cohort = "GSE78104"

# Input paths
in_trait_dir = "../../input/GEO/Anxiety_disorder"
in_cohort_dir = "../../input/GEO/Anxiety_disorder/GSE78104"

# Output paths
out_data_file = "../../output/preprocess/Anxiety_disorder/GSE78104.csv"
out_gene_data_file = "../../output/preprocess/Anxiety_disorder/gene_data/GSE78104.csv"
out_clinical_data_file = "../../output/preprocess/Anxiety_disorder/clinical_data/GSE78104.csv"
json_path = "../../output/preprocess/Anxiety_disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"lncRNA and mRNA expression data in peripheral blood sampled from patients with Obsessive-Compulsive Disorder"
!Series_summary	"The aim of the study is to identify the global messenger RNA (mRNA) and long noncoding RNA (lncRNA) expression profiling in peripheral blood from thirty patients with Obsessive Compulsive Disorders (OCD) and thirty paired normal controls."
!Series_overall_design	"We quantified the gene transcripts in peripheral blood from thirty patients with OCD and thirty normal controls by the method of Microarray using Aglilent G3 lncRNA v4.04×180K."
Sample Characteristics Dictionary:
{0: ['tissue: whole blood'], 1: ['disease state: Obsessive-Compulsive Disorder', 'disease state: normal control'], 2: ['gender: male', 'gender: female'], 3: ['age: 25y', 'age: 23y', 'age: 18y', 'age: 26y', 'age: 27y', 'age: 19y', 'age: 22y', 'age: 16y', 'age: 35y', 'age: 32y', 'age: 15y', 'age: 43y', 'age: 36y', 'age: 17y', 'age: 45y', 'age: 40y', 'age: 2

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains both mRNA and lncRNA expression data,
# which are suitable for our gene expression analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the data, Obsessive-Compulsive Disorder is in sample characteristic row 1
# We need to map this to Anxiety_disorder as per our study focus
trait_row = 1
# Age is in sample characteristic row 3
age_row = 3
# Gender is in sample characteristic row 2
gender_row = 2

# 2.2 Data Type Conversion
# For trait, treat OCD as related to anxiety disorder based on clinical knowledge
def convert_trait(value):
    if not value or ":" not in value:
        return None
    value = value.split(":", 1)[1].strip().lower()
    if "obsessive-compulsive disorder" in value or "ocd" in value:
        # OCD is considered an anxiety-related disorder in this study
        return 1
    elif "normal control" in value or "control" in value or "healthy" in value:
        return 0
    return None

# For age, convert to continuous numeric values
def convert_age(value):
    if not value or ":" not in value:
        return None
    value = value.split(":", 1)[1].strip()
    # Extract digits from strings like "age: 25y"
    import re
    match = re.search(r'(\d+)', value)
    if match:
        return int(match.group(1))
    return None

# For gender, convert to binary (female: 0, male: 1)
def convert_gender(value):
    if not value or ":" not in value:
        return None
    value = value.split(":", 1)[1].strip().lower()
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    return None

# 3. Save Metadata
# Trait data is available as trait_row is not None
is_trait_available = trait_row is not None
# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We need to access the raw sample characteristics data
    # First, try to load from the expected location
    clinical_file_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    
    # Check if the file exists before attempting to read it
    if os.path.exists(clinical_file_path):
        clinical_data = pd.read_csv(clinical_file_path)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the data
        print("Preview of selected clinical features:")
        print(preview_df(selected_clinical_df))
        
        # Save clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    else:
        print(f"Clinical data file not found at {clinical_file_path}")
        print("Unable to proceed with clinical feature extraction.")
        # Create empty or default clinical data to allow the pipeline to continue
        print("Creating empty clinical data structure for compatibility.")
        # This empty dataframe will be handled in subsequent steps
        empty_clinical_df = pd.DataFrame(columns=[trait, 'Age', 'Gender'])
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        empty_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Empty clinical data template saved to {out_clinical_data_file}")


Clinical data file not found at ../../input/GEO/Anxiety_disorder/GSE78104/clinical_data.csv
Unable to proceed with clinical feature extraction.
Creating empty clinical data structure for compatibility.
Empty clinical data template saved to ../../output/preprocess/Anxiety_disorder/clinical_data/GSE78104.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '(-)3xSLv1', 'A_19_P00315459',
       'A_19_P00315492', 'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315538',
       'A_19_P00315633', 'A_19_P00315668', 'A_19_P00315717', 'A_19_P00315718'],
      dtype='object', name='ID')

Gene data dimensions: 111087 genes × 60 samples


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers:
# These appear to be platform-specific probe IDs rather than standard human gene symbols.
# The identifiers like "(+)E1A_r60_1" and "A_19_P00315459" are not standard gene symbols
# but rather appear to be Agilent microarray probe IDs.
# Standard human gene symbols would look like BRCA1, TP53, etc.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_19_P00315459', 'A_19_P00315492', 'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315538'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'SEQUENCE': ['AGCCCCCACTGTTCCACTTATTGTGATGGTTTGTATATCTTTATTTCAAAGAAGATCTGT', 'AGGCAGCCTTGCTGTTGGGGGTTATTGGCAGCTGTTGGGGGTTAGAGACAGGACTCTCAT', 'AGCCGGGATCGGGTTGTTGTTAATTTCTTAAGCAATTTCTAAATTCTGTATTGACTCTCT', 'CAATGGATTCCATGTTTCTTTTTCTTGGGGGGAGCAGGGAGGGAGAAAGGTAGAAAAATG', 'CACAATGACCATCATTGAGGGCGATGTTTATGCTTCCATTGTTAGTTTAGATATTTTGTT'], 'TargetID': [nan, 'Q73P46', 'P01115', nan, nan], 'ncRNA_SeqID': [nan, nan, nan, nan, nan], 'Source': ['Agilent_humanG3V2', 'Agilent_humanG3V2', 'Agilent_humanG3V2', nan, nan], 'ncRNA_Accession': [nan, nan, nan, nan, nan], 'Chr': ['chrX', 'chr4', 'chr10', nan, nan], 'Start': [149131107.0, 129376376.0, 6780785.0, nan, nan], 'End': [149131166.0, 129376435.0, 6780844.0, nan, nan], 'strand': ['+', '+', '+', nan, nan], 'Description': [nan, 'Q73P46_TREDE (Q73P46) Branched-chain ami

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the appropriate columns for gene mapping
# Looking at the gene annotation preview, I can see:
# - The 'ID' column contains the probe identifiers (e.g., 'A_19_P00315459')
# - The 'GeneSymbol' column appears to contain gene symbols, but seems to have some non-standard entries

# 2. Create the gene mapping dataframe
# The mapping should be from 'ID' (probe identifier) to 'GeneSymbol' (gene symbol)
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GeneSymbol')

# Print some information about the mapping
print(f"Gene mapping shape: {mapping_df.shape}")
print("First few rows of the mapping data:")
print(mapping_df.head())

# Check how many unique probe IDs and gene symbols exist in the mapping
print(f"Number of unique probe IDs in mapping: {mapping_df['ID'].nunique()}")
print(f"Number of unique gene symbols in mapping: {mapping_df['Gene'].nunique()}")

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# Use the function that handles the many-to-many relation between probes and genes
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)

# Print information about the resulting gene expression data
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols in the processed data:")
print(gene_data.index[:10])

# Additional check to ensure we have meaningful gene symbols
unique_genes = gene_data.index.unique()
print(f"Number of unique genes after mapping: {len(unique_genes)}")
print(f"Example gene symbols: {list(unique_genes[:20])}")


Gene mapping shape: (21812, 2)
First few rows of the mapping data:
               ID     Gene
1  A_19_P00315492   Q73P46
2  A_19_P00315502   P01115
6  A_19_P00315668    HIPK2
7  A_19_P00315717  FAM200B
8  A_19_P00315718  FAM200B
Number of unique probe IDs in mapping: 21812
Number of unique gene symbols in mapping: 16487
Gene expression data shape after mapping: (15681, 60)
First few gene symbols in the processed data:
Index(['A1BG', 'A1CF', 'A2LD1', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS',
       'AADAC', 'AADACL2'],
      dtype='object', name='Gene')
Number of unique genes after mapping: 15681
Example gene symbols: ['A1BG', 'A1CF', 'A2LD1', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AADACL2', 'AADACL3', 'AADACL4', 'AAGAB', 'AAK1', 'AAMP', 'AANAT', 'AARS', 'AARS2', 'AASDHPPT', 'AATF']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 normalized gene symbols: {normalized_gene_data.index[:5].tolist()}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Re-extract clinical data since step 2 identified that trait data is available
# First, get the paths again
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get background information and clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Extract clinical features using the conversion functions defined in step 2
def convert_trait(value):
    if not value or ":" not in value:
        return None
    value = value.split(":", 1)[1].strip().lower()
    if "obsessive-compulsive disorder" in value or "ocd" in value:
        # OCD is considered an anxiety-related disorder in this study
        return 1
    elif "normal control" in value or "control" in value or "healthy" in value:
        return 0
    return None

def convert_age(value):
    if not value or ":" not in value:
        return None
    value = value.split(":", 1)[1].strip()
    import re
    match = re.search(r'(\d+)', value)
    if match:
        return int(match.group(1))
    return None

def convert_gender(value):
    if not value or ":" not in value:
        return None
    value = value.split(":", 1)[1].strip().lower()
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    return None

# Using values identified in step 2
trait_row = 1  # OCD status
age_row = 3    # Age
gender_row = 2 # Gender

# Extract clinical features
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct final quality validation
is_trait_available = True  # We confirmed trait data is available in step 2
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains human OCD data, which is relevant to anxiety disorders. Contains gene expression, age, and gender information."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: (15442, 60)
First 5 normalized gene symbols: ['A1BG', 'A1CF', 'A2M', 'A4GALT', 'A4GNT']


Normalized gene data saved to ../../output/preprocess/Anxiety_disorder/gene_data/GSE78104.csv


Clinical data saved to ../../output/preprocess/Anxiety_disorder/clinical_data/GSE78104.csv
Linked data shape: (60, 15445)


Data shape after handling missing values: (60, 15445)
For the feature 'Anxiety_disorder', the least common label is '1.0' with 30 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Anxiety_disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 18.75
  50% (Median): 27.0
  75%: 35.0
Min: 15.0
Max: 60.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 20 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Anxiety_disorder/GSE78104.csv
